In [13]:
#!pip install pandas
import cv2
import os
import csv
import numpy as np
import mediapipe as mp
import pandas as pd 

In [14]:
def create_pose_csv(cap, create_csv):
    ''' Create pose detections csv  with a video.'''
    if (cap.isOpened() == False):
        print("\nError opening the video file.")
        return
    else:
        pass
    # Color difine
    color_pose1 = (245,117,66)
    color_pose2 = (245,66,230)

    mp_drawing = mp.solutions.drawing_utils # Drawing helpers.
    mp_holistic = mp.solutions.holistic     # Mediapipe Solutions.

    # Initiate holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, frame = cap.read()
            if ret == True:
                # Recolor Feed
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False        

                # Make Detections
                results = holistic.process(image)

                # Recolor image back to BGR for rendering
                image.flags.writeable = True   
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                
                # Pose Detections
                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                    mp_drawing.DrawingSpec(color=color_pose1, thickness=2, circle_radius=4),
                    mp_drawing.DrawingSpec(color=color_pose2, thickness=2, circle_radius=2)
                )

                try:
                    num_coords = len(results.pose_landmarks.landmark) # num_coords: 33

                    landmarks = ['class'] # Create first rows data.
                    for val in range(1, num_coords+1):
                        landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]
                    
                    # E.g., (pose+face)2005=1+501*4, (pose+r_hand)217=1+54*4, 133=1+33*4
                    # print(f'len(landmarks): {len(landmarks)}')

                    # Define first class rows in csv file.
                    with open(create_csv, mode='w', newline='') as f:
                        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                        csv_writer.writerow(landmarks)
                except:
                    pass

                cv2.imshow('Raw Video Feed', image)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            else:
                break

    print(f'\nCreate {dataset_csv_file} done! \n\nNow you can run again.')
    cap.release()
    cv2.destroyAllWindows()



In [15]:
def add_record_coordinates(cap, class_name, export_csv):
    if (cap.isOpened() == False):
        print("Error opening the video file.")
    else:
        input_fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(f'Frames per second: {input_fps}')
        print(f'Frame count: {frame_count}')

    mp_drawing = mp.solutions.drawing_utils # Drawing helpers.
    mp_holistic = mp.solutions.holistic     # Mediapipe Solutions.

    # Initiate holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        
        while cap.isOpened():
            ret, frame = cap.read()
            if ret == True:
                # Recolor Feed
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False        

                # Make Detections
                results = holistic.process(image)
                # print(results.face_landmarks)

                # Recolor image back to BGR for rendering
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                #  Pose Detections
                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                    mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=4),
                    mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                )
                # Export coordinates
                try:
                    # Extract Pose landmarks
                    pose = results.pose_landmarks.landmark
                    pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

                    # Extract Face landmarks
                    # face = results.face_landmarks.landmark
                    row = pose_row

                    # Append class name.
                    row.insert(0, class_name)

                    # Export to CSV
                    with open(export_csv, mode='a', newline='') as f:
                        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                        csv_writer.writerow(row) 

                except:
                    pass

                cv2.imshow('Raw Webcam Feed', image)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            else:
                break
            
    print('Add done!\n -------------------')
    cap.release()
    cv2.destroyAllWindows()
    # check_csv_contents(file=export_csv)


In [16]:
# check_csv_contents(file=export_csv)
def check_csv_contents(file):
    df = pd.read_csv(file)
    print(f'Top5 datas: \n{df.head()}')
    print(f'Last5 datas: \n{df.tail()}')

In [17]:
class_catagory=0
# Can create train dataset or test dataset.
dataset_csv_file = 'data.csv'

video = "../video/0.cat-cow.mp4"

cap = cv2.VideoCapture(video)

if os.path.isfile(dataset_csv_file):
    print (f'{dataset_csv_file}: Exist.')
    print(f'class Name: {class_catagory} \n-----------------')

    add_record_coordinates(cap=cap, class_name=class_catagory, export_csv=dataset_csv_file)
else:
        print (f'{dataset_csv_file}: Not exist.')
        print('\nInitiate creating a csv file....\n')
        create_pose_csv(cap, create_csv=dataset_csv_file)

data.csv: Exist.
class Name: 0 
-----------------
Frames per second: 25.0
Frame count: 714
Add done!
 -------------------


In [1]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Normalization

In [ ]:
class CsvDataset:
    def __init__(self, file):
        self.dataframe = pd.read_csv(file)
        self.val_df = None
        self.train_df = None
        self.val_ds = None
        self.train_ds = None

    def csv_preprocessing(self):
        
        df2 = self.dataframe.copy()

        columns_removed = [
            'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11',
            'y1', 'y2', 'y3', 'y4', 'y5', 'y6', 'y7', 'y8', 'y9', 'y10', 'y11',
            'z1', 'z2', 'z3', 'z4', 'z5', 'z6', 'z7', 'z8', 'z9', 'z10', 'z11',
            'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11',
        ]

        df2 = df2.drop(columns_removed, axis = 'columns')
        return df2
    
    def df_to_datasets(self, dataframe, target):
        # 
        self.val_df = dataframe.sample(frac=0.2, random_state=1337)
        # drop the colum 1 of 'class'.
        self.train_df = dataframe.drop(self.val_df.index)

        train_df = self.train_df.copy()
        val_df = self.val_df.copy()

        train_labels = train_df.pop(target)
        val_labels = val_df.pop(target)
        
        self.train_ds = tf.data.Dataset.from_tensor_slices((dict(train_df), train_labels))
        self.val_ds = tf.data.Dataset.from_tensor_slices((dict(val_df), val_labels))
        self.train_ds = self.train_ds.shuffle(buffer_size=len(self.train_ds))
        self.val_ds = self.val_ds.shuffle(buffer_size=len(self.val_ds))

        return self.train_ds, self.val_ds


class EncodeFeatures:

    def __init__(self):
        self.feature_ds = None
            
    def numerical_feature(self, feature, name, dataset):
        # Create a Normalization layer for our feature
        normalizer = Normalization()

        # Prepare a Dataset that only yields our feature
        self.feature_ds = dataset.map(lambda x, y: x[name])
        self.feature_ds = self.feature_ds.map(lambda x: tf.expand_dims(x, -1))

        # Learn the statistics of the data
        normalizer.adapt(self.feature_ds)

        # Normalize the input feature
        encoded_feature = normalizer(feature)
        return encoded_feature


def point():
    x = [
        'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22',
        'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29', 'x30', 'x31', 'x32', 'x33',
    ]
    y = [
        'y12', 'y13', 'y14', 'y15', 'y16', 'y17', 'y18', 'y19', 'y20', 'y21', 'y22',
        'y23', 'y24', 'y25', 'y26', 'y27', 'y28', 'y29', 'y30', 'y31', 'y32', 'y33',
    ]
    z = [
        'z12', 'z13', 'z14', 'z15', 'z16', 'z17', 'z18', 'z19', 'z20', 'z21', 'z22',
        'z23', 'z24', 'z25', 'z26', 'z27', 'z28', 'z29', 'z30', 'z31', 'z32', 'z33',
    ]
    v = [
        'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19', 'v20', 'v21', 'v22',
        'v23', 'v24', 'v25', 'v26', 'v27', 'v28', 'v29', 'v30', 'v31', 'v32', 'v33',
    ]
    coords = [x, y, z, v]
    return coords


def input_features():

    coords = point()
    # point 12 ~ 33
    # Numerical features
    x12 = keras.Input(shape=(1,), name=coords[0][0])
    x13 = keras.Input(shape=(1,), name=coords[0][1])
    x14 = keras.Input(shape=(1,), name=coords[0][2])
    x15 = keras.Input(shape=(1,), name=coords[0][3])
    x16 = keras.Input(shape=(1,), name=coords[0][4])
    x17 = keras.Input(shape=(1,), name=coords[0][5]) 
    x18 = keras.Input(shape=(1,), name=coords[0][6])
    x19 = keras.Input(shape=(1,), name=coords[0][7])
    x20 = keras.Input(shape=(1,), name=coords[0][8])
    x21 = keras.Input(shape=(1,), name=coords[0][9])
    x22 = keras.Input(shape=(1,), name=coords[0][10])
    x23 = keras.Input(shape=(1,), name=coords[0][11])
    x24 = keras.Input(shape=(1,), name=coords[0][12])
    x25 = keras.Input(shape=(1,), name=coords[0][13])
    x26 = keras.Input(shape=(1,), name=coords[0][14])
    x27 = keras.Input(shape=(1,), name=coords[0][15])
    x28 = keras.Input(shape=(1,), name=coords[0][16])
    x29 = keras.Input(shape=(1,), name=coords[0][17])
    x30 = keras.Input(shape=(1,), name=coords[0][18])
    x31 = keras.Input(shape=(1,), name=coords[0][19])
    x32 = keras.Input(shape=(1,), name=coords[0][20])
    x33 = keras.Input(shape=(1,), name=coords[0][21])

    y12 = keras.Input(shape=(1,), name=coords[1][0])
    y13 = keras.Input(shape=(1,), name=coords[1][1])
    y14 = keras.Input(shape=(1,), name=coords[1][2])
    y15 = keras.Input(shape=(1,), name=coords[1][3])
    y16 = keras.Input(shape=(1,), name=coords[1][4])
    y17 = keras.Input(shape=(1,), name=coords[1][5]) 
    y18 = keras.Input(shape=(1,), name=coords[1][6])
    y19 = keras.Input(shape=(1,), name=coords[1][7])
    y20 = keras.Input(shape=(1,), name=coords[1][8])
    y21 = keras.Input(shape=(1,), name=coords[1][9])
    y22 = keras.Input(shape=(1,), name=coords[1][10])
    y23 = keras.Input(shape=(1,), name=coords[1][11])
    y24 = keras.Input(shape=(1,), name=coords[1][12])
    y25 = keras.Input(shape=(1,), name=coords[1][13])
    y26 = keras.Input(shape=(1,), name=coords[1][14])
    y27 = keras.Input(shape=(1,), name=coords[1][15])
    y28 = keras.Input(shape=(1,), name=coords[1][16])
    y29 = keras.Input(shape=(1,), name=coords[1][17])
    y30 = keras.Input(shape=(1,), name=coords[1][18])
    y31 = keras.Input(shape=(1,), name=coords[1][19])
    y32 = keras.Input(shape=(1,), name=coords[1][20])
    y33 = keras.Input(shape=(1,), name=coords[1][21])

    z12 = keras.Input(shape=(1,), name=coords[2][0])
    z13 = keras.Input(shape=(1,), name=coords[2][1])
    z14 = keras.Input(shape=(1,), name=coords[2][2])
    z15 = keras.Input(shape=(1,), name=coords[2][3])
    z16 = keras.Input(shape=(1,), name=coords[2][4])
    z17 = keras.Input(shape=(1,), name=coords[2][5]) 
    z18 = keras.Input(shape=(1,), name=coords[2][6])
    z19 = keras.Input(shape=(1,), name=coords[2][7])
    z20 = keras.Input(shape=(1,), name=coords[2][8])
    z21 = keras.Input(shape=(1,), name=coords[2][9])
    z22 = keras.Input(shape=(1,), name=coords[2][10])
    z23 = keras.Input(shape=(1,), name=coords[2][11])
    z24 = keras.Input(shape=(1,), name=coords[2][12])
    z25 = keras.Input(shape=(1,), name=coords[2][13])
    z26 = keras.Input(shape=(1,), name=coords[2][14])
    z27 = keras.Input(shape=(1,), name=coords[2][15])
    z28 = keras.Input(shape=(1,), name=coords[2][16])
    z29 = keras.Input(shape=(1,), name=coords[2][17])
    z30 = keras.Input(shape=(1,), name=coords[2][18])
    z31 = keras.Input(shape=(1,), name=coords[2][19])
    z32 = keras.Input(shape=(1,), name=coords[2][20])
    z33 = keras.Input(shape=(1,), name=coords[2][21])

    v12 = keras.Input(shape=(1,), name=coords[3][0])
    v13 = keras.Input(shape=(1,), name=coords[3][1])
    v14 = keras.Input(shape=(1,), name=coords[3][2])
    v15 = keras.Input(shape=(1,), name=coords[3][3])
    v16 = keras.Input(shape=(1,), name=coords[3][4])
    v17 = keras.Input(shape=(1,), name=coords[3][5]) 
    v18 = keras.Input(shape=(1,), name=coords[3][6])
    v19 = keras.Input(shape=(1,), name=coords[3][7])
    v20 = keras.Input(shape=(1,), name=coords[3][8])
    v21 = keras.Input(shape=(1,), name=coords[3][9])
    v22 = keras.Input(shape=(1,), name=coords[3][10])
    v23 = keras.Input(shape=(1,), name=coords[3][11])
    v24 = keras.Input(shape=(1,), name=coords[3][12])
    v25 = keras.Input(shape=(1,), name=coords[3][13])
    v26 = keras.Input(shape=(1,), name=coords[3][14])
    v27 = keras.Input(shape=(1,), name=coords[3][15])
    v28 = keras.Input(shape=(1,), name=coords[3][16])
    v29 = keras.Input(shape=(1,), name=coords[3][17])
    v30 = keras.Input(shape=(1,), name=coords[3][18])
    v31 = keras.Input(shape=(1,), name=coords[3][19])
    v32 = keras.Input(shape=(1,), name=coords[3][20])
    v33 = keras.Input(shape=(1,), name=coords[3][21])

    all_inputs = [
        x12, x13, x14, x15, x16, x17, x18, x19, x20, x21, x22, 
        x23, x24, x25, x26, x27, x28, x29, x30, x31, x32, x33,

        y12, y13, y14, y15, y16, y17, y18, y19, y20, y21, y22, 
        y23, y24, y25, y26, y27, y28, y29, y30, y31, y32, y33,

        z12, z13, z14, z15, z16, z17, z18, z19, z20, z21, z22, 
        z23, z24, z25, z26, z27, z28, z29, z30, z31, z32, z33,

        v12, v13, v14, v15, v16, v17, v18, v19, v20, v21, v22, 
        v23, v24, v25, v26, v27, v28, v29, v30, v31, v32, v33,
    ]

    return all_inputs


def specify_encoded_features(train_ds, all_inputs):

    encoded = EncodeFeatures()
    coords_p = point()

    # point 12 ~ 33
    # Numerical features
    x12 = encoded.numerical_feature(all_inputs[0], coords_p[0][0], train_ds)
    x13 = encoded.numerical_feature(all_inputs[1], coords_p[0][1], train_ds)
    x14 = encoded.numerical_feature(all_inputs[2], coords_p[0][2], train_ds)
    x15 = encoded.numerical_feature(all_inputs[3], coords_p[0][3], train_ds)
    x16 = encoded.numerical_feature(all_inputs[4], coords_p[0][4], train_ds)
    x17 = encoded.numerical_feature(all_inputs[5], coords_p[0][5], train_ds)
    x18 = encoded.numerical_feature(all_inputs[6], coords_p[0][6], train_ds)
    x19 = encoded.numerical_feature(all_inputs[7], coords_p[0][7], train_ds)
    x20 = encoded.numerical_feature(all_inputs[8], coords_p[0][8], train_ds)
    x21 = encoded.numerical_feature(all_inputs[9], coords_p[0][9], train_ds)
    x22 = encoded.numerical_feature(all_inputs[10], coords_p[0][10], train_ds)
    x23 = encoded.numerical_feature(all_inputs[11], coords_p[0][11], train_ds)
    x24 = encoded.numerical_feature(all_inputs[12], coords_p[0][12], train_ds)
    x25 = encoded.numerical_feature(all_inputs[13], coords_p[0][13], train_ds)
    x26 = encoded.numerical_feature(all_inputs[14], coords_p[0][14], train_ds)
    x27 = encoded.numerical_feature(all_inputs[15], coords_p[0][15], train_ds)
    x28 = encoded.numerical_feature(all_inputs[16], coords_p[0][16], train_ds)
    x29 = encoded.numerical_feature(all_inputs[17], coords_p[0][17], train_ds)
    x30 = encoded.numerical_feature(all_inputs[18], coords_p[0][18], train_ds)
    x31 = encoded.numerical_feature(all_inputs[19], coords_p[0][19], train_ds)
    x32 = encoded.numerical_feature(all_inputs[20], coords_p[0][20], train_ds)
    x33 = encoded.numerical_feature(all_inputs[21], coords_p[0][21], train_ds)

    y12 = encoded.numerical_feature(all_inputs[22], coords_p[1][0], train_ds)
    y13 = encoded.numerical_feature(all_inputs[23], coords_p[1][1], train_ds)
    y14 = encoded.numerical_feature(all_inputs[24], coords_p[1][2], train_ds)
    y15 = encoded.numerical_feature(all_inputs[25], coords_p[1][3], train_ds)
    y16 = encoded.numerical_feature(all_inputs[26], coords_p[1][4], train_ds)
    y17 = encoded.numerical_feature(all_inputs[27], coords_p[1][5], train_ds)
    y18 = encoded.numerical_feature(all_inputs[28], coords_p[1][6], train_ds)
    y19 = encoded.numerical_feature(all_inputs[29], coords_p[1][7], train_ds)
    y20 = encoded.numerical_feature(all_inputs[30], coords_p[1][8], train_ds)
    y21 = encoded.numerical_feature(all_inputs[31], coords_p[1][9], train_ds)
    y22 = encoded.numerical_feature(all_inputs[32], coords_p[1][10], train_ds)
    y23 = encoded.numerical_feature(all_inputs[33], coords_p[1][11], train_ds)
    y24 = encoded.numerical_feature(all_inputs[34], coords_p[1][12], train_ds)
    y25 = encoded.numerical_feature(all_inputs[35], coords_p[1][13], train_ds)
    y26 = encoded.numerical_feature(all_inputs[36], coords_p[1][14], train_ds)
    y27 = encoded.numerical_feature(all_inputs[37], coords_p[1][15], train_ds)
    y28 = encoded.numerical_feature(all_inputs[38], coords_p[1][16], train_ds)
    y29 = encoded.numerical_feature(all_inputs[39], coords_p[1][17], train_ds)
    y30 = encoded.numerical_feature(all_inputs[40], coords_p[1][18], train_ds)
    y31 = encoded.numerical_feature(all_inputs[41], coords_p[1][19], train_ds)
    y32 = encoded.numerical_feature(all_inputs[42], coords_p[1][20], train_ds)
    y33 = encoded.numerical_feature(all_inputs[43], coords_p[1][21], train_ds)

    z12 = encoded.numerical_feature(all_inputs[44], coords_p[2][0], train_ds)
    z13 = encoded.numerical_feature(all_inputs[45], coords_p[2][1], train_ds)
    z14 = encoded.numerical_feature(all_inputs[46], coords_p[2][2], train_ds)
    z15 = encoded.numerical_feature(all_inputs[47], coords_p[2][3], train_ds)
    z16 = encoded.numerical_feature(all_inputs[48], coords_p[2][4], train_ds)
    z17 = encoded.numerical_feature(all_inputs[49], coords_p[2][5], train_ds)
    z18 = encoded.numerical_feature(all_inputs[50], coords_p[2][6], train_ds)
    z19 = encoded.numerical_feature(all_inputs[51], coords_p[2][7], train_ds)
    z20 = encoded.numerical_feature(all_inputs[52], coords_p[2][8], train_ds)
    z21 = encoded.numerical_feature(all_inputs[53], coords_p[2][9], train_ds)
    z22 = encoded.numerical_feature(all_inputs[54], coords_p[2][10], train_ds)
    z23 = encoded.numerical_feature(all_inputs[55], coords_p[2][11], train_ds)
    z24 = encoded.numerical_feature(all_inputs[56], coords_p[2][12], train_ds)
    z25 = encoded.numerical_feature(all_inputs[57], coords_p[2][13], train_ds)
    z26 = encoded.numerical_feature(all_inputs[58], coords_p[2][14], train_ds)
    z27 = encoded.numerical_feature(all_inputs[59], coords_p[2][15], train_ds)
    z28 = encoded.numerical_feature(all_inputs[60], coords_p[2][16], train_ds)
    z29 = encoded.numerical_feature(all_inputs[61], coords_p[2][17], train_ds)
    z30 = encoded.numerical_feature(all_inputs[62], coords_p[2][18], train_ds)
    z31 = encoded.numerical_feature(all_inputs[63], coords_p[2][19], train_ds)
    z32 = encoded.numerical_feature(all_inputs[64], coords_p[2][20], train_ds)
    z33 = encoded.numerical_feature(all_inputs[65], coords_p[2][21], train_ds)

    v12 = encoded.numerical_feature(all_inputs[66], coords_p[3][0], train_ds)
    v13 = encoded.numerical_feature(all_inputs[67], coords_p[3][1], train_ds)
    v14 = encoded.numerical_feature(all_inputs[68], coords_p[3][2], train_ds)
    v15 = encoded.numerical_feature(all_inputs[69], coords_p[3][3], train_ds)
    v16 = encoded.numerical_feature(all_inputs[70], coords_p[3][4], train_ds)
    v17 = encoded.numerical_feature(all_inputs[71], coords_p[3][5], train_ds)
    v18 = encoded.numerical_feature(all_inputs[72], coords_p[3][6], train_ds)
    v19 = encoded.numerical_feature(all_inputs[73], coords_p[3][7], train_ds)
    v20 = encoded.numerical_feature(all_inputs[74], coords_p[3][8], train_ds)
    v21 = encoded.numerical_feature(all_inputs[75], coords_p[3][9], train_ds)
    v22 = encoded.numerical_feature(all_inputs[76], coords_p[3][10], train_ds)
    v23 = encoded.numerical_feature(all_inputs[77], coords_p[3][11], train_ds)
    v24 = encoded.numerical_feature(all_inputs[78], coords_p[3][12], train_ds)
    v25 = encoded.numerical_feature(all_inputs[79], coords_p[3][13], train_ds)
    v26 = encoded.numerical_feature(all_inputs[80], coords_p[3][14], train_ds)
    v27 = encoded.numerical_feature(all_inputs[81], coords_p[3][15], train_ds)
    v28 = encoded.numerical_feature(all_inputs[82], coords_p[3][16], train_ds)
    v29 = encoded.numerical_feature(all_inputs[83], coords_p[3][17], train_ds)
    v30 = encoded.numerical_feature(all_inputs[84], coords_p[3][18], train_ds)
    v31 = encoded.numerical_feature(all_inputs[85], coords_p[3][19], train_ds)
    v32 = encoded.numerical_feature(all_inputs[86], coords_p[3][20], train_ds)
    v33 = encoded.numerical_feature(all_inputs[87], coords_p[3][21], train_ds)

    all_features = layers.concatenate(
        [
            x12, x13, x14, x15, x16, x17, x18, x19, x20, x21, x22, 
            x23, x24, x25, x26, x27, x28, x29, x30, x31, x32, x33,

            y12, y13, y14, y15, y16, y17, y18, y19, y20, y21, y22,
            y23, y24, y25, y26, y27, y28, y29, y30, y31, y32, y33,

            z12, z13, z14, z15, z16, z17, z18, z19, z20, z21, z22, 
            z23, z24, z25, z26, z27, z28, z29, z30, z31, z32, z33,

            v12, v13, v14, v15, v16, v17, v18, v19, v20, v21, v22,
            v23, v24, v25, v26, v27, v28, v29, v30, v31, v32, v33,
        ]
    )
    return all_features


In [6]:

dataset_csv_file = 'data.csv'
target_value = 'class'
all_model = './model_weights/all_model/08.31_xyzv/3_categories_pose' # all_model: Model struct and model weights.

# Data preprocessed and creat datasets.
pose_datasets = CsvDataset(file=dataset_csv_file)
df_pose = pose_datasets.csv_preprocessing()
train_ds, val_ds = pose_datasets.df_to_datasets(dataframe=df_pose, target=target_value)



train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)

    # Functional API model build.
all_inputs = input_features()
all_features = specify_encoded_features(train_ds, all_inputs)
x = layers.Dense(32, activation='relu')(all_features)
x = layers.Dropout(0.3)(x)
    # output = layers.Dense(3, activation='sigmoid')(x) # from_logits=False
output = layers.Dense(6, activation='softmax')(x) # from_logits=False
model = keras.Model(all_inputs, output)

model.compile(optimizer='sgd', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

    # Model train.
model.fit(x=train_ds, epochs=100, verbose=2, validation_data=val_ds)

    # # Option1: Save Keras model. # #
    # model.save(all_model)
    # print('Model save done!')

    # # Option2: Save TFLite model. # #
    # Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
# Save the model.
with open('softmaxmodel.tflite', 'wb') as f:
    f.write(tflite_model)

print('TFLite Model save done!')

Epoch 1/100
132/132 - 6s - loss: 0.9913 - accuracy: 0.6349 - val_loss: 0.4120 - val_accuracy: 0.9164 - 6s/epoch - 46ms/step
Epoch 2/100
132/132 - 2s - loss: 0.4251 - accuracy: 0.8702 - val_loss: 0.2248 - val_accuracy: 0.9677 - 2s/epoch - 12ms/step
Epoch 3/100
132/132 - 2s - loss: 0.3034 - accuracy: 0.9200 - val_loss: 0.1525 - val_accuracy: 0.9905 - 2s/epoch - 12ms/step
Epoch 4/100
132/132 - 2s - loss: 0.2330 - accuracy: 0.9445 - val_loss: 0.1097 - val_accuracy: 0.9934 - 2s/epoch - 12ms/step
Epoch 5/100
132/132 - 2s - loss: 0.1971 - accuracy: 0.9499 - val_loss: 0.0849 - val_accuracy: 0.9943 - 2s/epoch - 12ms/step
Epoch 6/100
132/132 - 2s - loss: 0.1642 - accuracy: 0.9649 - val_loss: 0.0674 - val_accuracy: 0.9953 - 2s/epoch - 12ms/step
Epoch 7/100
132/132 - 2s - loss: 0.1446 - accuracy: 0.9661 - val_loss: 0.0540 - val_accuracy: 0.9962 - 2s/epoch - 12ms/step
Epoch 8/100
132/132 - 2s - loss: 0.1306 - accuracy: 0.9694 - val_loss: 0.0446 - val_accuracy: 0.9962 - 2s/epoch - 12ms/step
Epoch 9/

TFLite Model save done!


# 3. Try to predict some exercise


In [1]:
#Model interface
from IPython.core.events import post_execute
import tensorflow as tf 
from tensorflow import keras
import cv2 
import mediapipe as mp
import pandas as pd
import numpy as np

2022-05-15 01:23:04.089312: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-15 01:23:04.089351: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [12]:

def point():
    x = [
        'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22',
        'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29', 'x30', 'x31', 'x32', 'x33',
    ]
    y = [
        'y12', 'y13', 'y14', 'y15', 'y16', 'y17', 'y18', 'y19', 'y20', 'y21', 'y22',
        'y23', 'y24', 'y25', 'y26', 'y27', 'y28', 'y29', 'y30', 'y31', 'y32', 'y33',
    ]
    z = [
        'z12', 'z13', 'z14', 'z15', 'z16', 'z17', 'z18', 'z19', 'z20', 'z21', 'z22',
        'z23', 'z24', 'z25', 'z26', 'z27', 'z28', 'z29', 'z30', 'z31', 'z32', 'z33',
    ]
    v = [
        'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19', 'v20', 'v21', 'v22',
        'v23', 'v24', 'v25', 'v26', 'v27', 'v28', 'v29', 'v30', 'v31', 'v32', 'v33',
    ]
    coords = [x, y, z, v]
    return coords

def display_tflite_classify_pose(cap, model):
    if (cap.isOpened() == False):
        print("Error opening the video file.")
    else:
        input_fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(f'Frames per second: {input_fps}')
        print(f'Frame count: {frame_count}')

    mp_drawing = mp.solutions.drawing_utils # Drawing helpers.
    mp_holistic = mp.solutions.holistic     # Mediapipe Solutions.

    # Initiate holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        
        while cap.isOpened():
            ret, frame = cap.read()
            if ret == True:
                # Recolor Feed
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False    

                # Make Detections
                results = holistic.process(image)

                # Recolor image back to BGR for rendering
                image.flags.writeable = True   
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                # Pose Detections
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                        )
                # Export coordinates
                try:
                    # Extract Pose landmarks
                    pose = results.pose_landmarks.landmark
                    pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

                    # Concate rows
                    row = pose_row
                    # print(f'type: {type(row)}, \n{row}')

                    # point[11] - point[32] input to tflite model.
                    coords = point()
                    specify_float = 8

                    dict_p12_to_p33 = {
                        # x12 to x33
                        coords[0][0]:round(row[44], specify_float),
                        coords[0][1]:round(row[48], specify_float),
                        coords[0][2]:round(row[52], specify_float),
                        coords[0][3]:round(row[56], specify_float),
                        coords[0][4]:round(row[60], specify_float),
                        coords[0][5]:round(row[64], specify_float),
                        coords[0][6]:round(row[68], specify_float),
                        coords[0][7]:round(row[72], specify_float),
                        coords[0][8]:round(row[76], specify_float),
                        coords[0][9]:round(row[80], specify_float),
                        coords[0][10]:round(row[84], specify_float),
                        coords[0][11]:round(row[88], specify_float),
                        coords[0][12]:round(row[92], specify_float),
                        coords[0][13]:round(row[96], specify_float),
                        coords[0][14]:round(row[100], specify_float),
                        coords[0][15]:round(row[104], specify_float),
                        coords[0][16]:round(row[108], specify_float),
                        coords[0][17]:round(row[112], specify_float),
                        coords[0][18]:round(row[116], specify_float),
                        coords[0][19]:round(row[120], specify_float),
                        coords[0][20]:round(row[124], specify_float),
                        coords[0][21]:round(row[128], specify_float),

                        # y12 to y33
                        coords[1][0]:round(row[45], specify_float),
                        coords[1][1]:round(row[49], specify_float),
                        coords[1][2]:round(row[53], specify_float),
                        coords[1][3]:round(row[57], specify_float),
                        coords[1][4]:round(row[61], specify_float),
                        coords[1][5]:round(row[65], specify_float),
                        coords[1][6]:round(row[69], specify_float),
                        coords[1][7]:round(row[73], specify_float),
                        coords[1][8]:round(row[77], specify_float),
                        coords[1][9]:round(row[81], specify_float),
                        coords[1][10]:round(row[85], specify_float),
                        coords[1][11]:round(row[89], specify_float),
                        coords[1][12]:round(row[93], specify_float),
                        coords[1][13]:round(row[97], specify_float),
                        coords[1][14]:round(row[101], specify_float),
                        coords[1][15]:round(row[105], specify_float),
                        coords[1][16]:round(row[109], specify_float),
                        coords[1][17]:round(row[113], specify_float),
                        coords[1][18]:round(row[117], specify_float),
                        coords[1][19]:round(row[121], specify_float),
                        coords[1][20]:round(row[125], specify_float),
                        coords[1][21]:round(row[129], specify_float),

                        # z12 to z33
                        coords[2][0]:round(row[46], specify_float),
                        coords[2][1]:round(row[50], specify_float),
                        coords[2][2]:round(row[54], specify_float),
                        coords[2][3]:round(row[58], specify_float),
                        coords[2][4]:round(row[62], specify_float),
                        coords[2][5]:round(row[66], specify_float),
                        coords[2][6]:round(row[70], specify_float),
                        coords[2][7]:round(row[74], specify_float),
                        coords[2][8]:round(row[78], specify_float),
                        coords[2][9]:round(row[82], specify_float),
                        coords[2][10]:round(row[86], specify_float),
                        coords[2][11]:round(row[90], specify_float),
                        coords[2][12]:round(row[94], specify_float),
                        coords[2][13]:round(row[98], specify_float),
                        coords[2][14]:round(row[102], specify_float),
                        coords[2][15]:round(row[106], specify_float),
                        coords[2][16]:round(row[110], specify_float),
                        coords[2][17]:round(row[114], specify_float),
                        coords[2][18]:round(row[118], specify_float),
                        coords[2][19]:round(row[122], specify_float),
                        coords[2][20]:round(row[126], specify_float),
                        coords[2][21]:round(row[130], specify_float),

                        # v12 to v33
                        coords[3][0]:round(row[47], specify_float),
                        coords[3][1]:round(row[51], specify_float),
                        coords[3][2]:round(row[55], specify_float),
                        coords[3][3]:round(row[59], specify_float),
                        coords[3][4]:round(row[63], specify_float),
                        coords[3][5]:round(row[67], specify_float),
                        coords[3][6]:round(row[71], specify_float),
                        coords[3][7]:round(row[75], specify_float),
                        coords[3][8]:round(row[79], specify_float),
                        coords[3][9]:round(row[83], specify_float),
                        coords[3][10]:round(row[87], specify_float),
                        coords[3][11]:round(row[91], specify_float),
                        coords[3][12]:round(row[95], specify_float),
                        coords[3][13]:round(row[99], specify_float),
                        coords[3][14]:round(row[103], specify_float),
                        coords[3][15]:round(row[107], specify_float),
                        coords[3][16]:round(row[111], specify_float),
                        coords[3][17]:round(row[115], specify_float),
                        coords[3][18]:round(row[119], specify_float),
                        coords[3][19]:round(row[123], specify_float),
                        coords[3][20]:round(row[127], specify_float),
                        coords[3][21]:round(row[131], specify_float),
                    }
                    input_dict = {name: np.expand_dims(np.array(value, dtype=np.float32), axis=0) for name, value in dict_p12_to_p33.items()}
                    
                    # Make Detections
                    # 0: cat_camel, 1: bridge_exercise, 2: heel_raise.
                    result = tflite_inference(input=input_dict, model=model)
                    body_language_class = np.argmax(result)
                    # body_language_prob = round(result[np.argmax(result)], 2)*100
                    body_language_prob = result[np.argmax(result)]

                    if str(body_language_class) == '0':
                        pose_class = 'Cat camel' 
                    elif str(body_language_class) == '1':
                        pose_class = 'Adho mukho Svanasana'
                    elif str(body_language_class) == '2':
                        pose_class = 'Setu bandh'
                    elif str(body_language_class) == '3':
                        pose_class = 'Tadasana'
                    elif str(body_language_class) == '4':
                        pose_class = 'Standing Oblique Crunch'
                    elif str(body_language_class) == '5':
                        pose_class = 'Uttansana'
                    else:
                        pose_class = 'detecting exercise'
                    
                    print(f'calss: {body_language_class}, prob: {body_language_prob}')

                    # Show pose category near the ear.
                    coords = tuple(np.multiply(
                        np.array(
                            (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                            results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)),
                        [1280,480]
                    ).astype(int))

                    # cv2.rectangle(影像, 頂點座標, 對向頂點座標, 顏色, 線條寬度).
                    # cv2.putText(影像, 文字, 座標, 字型, 大小, 顏色, 線條寬度, 線條種類).
                    cv2.rectangle(image, 
                                (coords[0], coords[1]+5), 
                                (coords[0]+200, coords[1]-30), 
                                (245, 117, 16), -1)
                    cv2.putText(image, pose_class, coords, 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                    # Get status box
                    cv2.rectangle(image, (10,0), (310, 55), (0, 0, 0), -1)

                    # Display Class
                    cv2.putText(
                        image, 
                        'CLASS: ', (15, 25), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.9, (255, 255, 0), 1, cv2.LINE_AA
                    )
                    cv2.putText(
                        image, 
                        pose_class, (120, 25), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        1, (255, 255, 255), 2, cv2.LINE_AA
                    )

                    # Display Probability
                    cv2.putText(
                        image, 
                        'PROB: ', (15, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.9, (255, 255, 0), 1, cv2.LINE_AA
                    )
                    cv2.putText(
                        image, 
                        str(body_language_prob), (120, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        1, (255, 255, 255), 2, cv2.LINE_AA
                    )

                except:
                    pass

                cv2.imshow('Raw Webcam Feed', image)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            
            else:
                break

    print('Done!')
    cap.release()
    cv2.destroyAllWindows()


def save_tflite_classify_pose(cap, model, result_out):
    if (cap.isOpened() == False):
        print("Error opening the video file.")
    else:
        input_fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        output_fps = input_fps - 1
        print(f'Frames per second: {input_fps}')
        print(f'Frame count: {frame_count}')

    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f'video_w: {w}, video_h: {h}')

    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # 輸出附檔名為 mp4. 
    out = cv2.VideoWriter(result_out, fourcc, output_fps, (w, h))

    mp_drawing = mp.solutions.drawing_utils # Drawing helpers.
    mp_holistic = mp.solutions.holistic     # Mediapipe Solutions.

    # Initiate holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        
        while cap.isOpened():
            ret, frame = cap.read()
            if ret == True:
                # Recolor Feed
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False    

                # Make Detections
                results = holistic.process(image)

                # Recolor image back to BGR for rendering
                image.flags.writeable = True   
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                # Pose Detections
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                        )
                # Export coordinates
                try:
                    # Extract Pose landmarks
                    pose = results.pose_landmarks.landmark
                    pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

                    # Concate rows
                    row = pose_row
                    # print(f'type: {type(row)}, \n{row}')

                    # point[11] - point[32] input to tflite model.
                    coords = point()
                    specify_float = 8

                    dict_p12_to_p33 = {
                        # x12 to x33
                        coords[0][0]:round(row[44], specify_float),
                        coords[0][1]:round(row[48], specify_float),
                        coords[0][2]:round(row[52], specify_float),
                        coords[0][3]:round(row[56], specify_float),
                        coords[0][4]:round(row[60], specify_float),
                        coords[0][5]:round(row[64], specify_float),
                        coords[0][6]:round(row[68], specify_float),
                        coords[0][7]:round(row[72], specify_float),
                        coords[0][8]:round(row[76], specify_float),
                        coords[0][9]:round(row[80], specify_float),
                        coords[0][10]:round(row[84], specify_float),
                        coords[0][11]:round(row[88], specify_float),
                        coords[0][12]:round(row[92], specify_float),
                        coords[0][13]:round(row[96], specify_float),
                        coords[0][14]:round(row[100], specify_float),
                        coords[0][15]:round(row[104], specify_float),
                        coords[0][16]:round(row[108], specify_float),
                        coords[0][17]:round(row[112], specify_float),
                        coords[0][18]:round(row[116], specify_float),
                        coords[0][19]:round(row[120], specify_float),
                        coords[0][20]:round(row[124], specify_float),
                        coords[0][21]:round(row[128], specify_float),

                        # y12 to y33
                        coords[1][0]:round(row[45], specify_float),
                        coords[1][1]:round(row[49], specify_float),
                        coords[1][2]:round(row[53], specify_float),
                        coords[1][3]:round(row[57], specify_float),
                        coords[1][4]:round(row[61], specify_float),
                        coords[1][5]:round(row[65], specify_float),
                        coords[1][6]:round(row[69], specify_float),
                        coords[1][7]:round(row[73], specify_float),
                        coords[1][8]:round(row[77], specify_float),
                        coords[1][9]:round(row[81], specify_float),
                        coords[1][10]:round(row[85], specify_float),
                        coords[1][11]:round(row[89], specify_float),
                        coords[1][12]:round(row[93], specify_float),
                        coords[1][13]:round(row[97], specify_float),
                        coords[1][14]:round(row[101], specify_float),
                        coords[1][15]:round(row[105], specify_float),
                        coords[1][16]:round(row[109], specify_float),
                        coords[1][17]:round(row[113], specify_float),
                        coords[1][18]:round(row[117], specify_float),
                        coords[1][19]:round(row[121], specify_float),
                        coords[1][20]:round(row[125], specify_float),
                        coords[1][21]:round(row[129], specify_float),

                        # z12 to z33
                        coords[2][0]:round(row[46], specify_float),
                        coords[2][1]:round(row[50], specify_float),
                        coords[2][2]:round(row[54], specify_float),
                        coords[2][3]:round(row[58], specify_float),
                        coords[2][4]:round(row[62], specify_float),
                        coords[2][5]:round(row[66], specify_float),
                        coords[2][6]:round(row[70], specify_float),
                        coords[2][7]:round(row[74], specify_float),
                        coords[2][8]:round(row[78], specify_float),
                        coords[2][9]:round(row[82], specify_float),
                        coords[2][10]:round(row[86], specify_float),
                        coords[2][11]:round(row[90], specify_float),
                        coords[2][12]:round(row[94], specify_float),
                        coords[2][13]:round(row[98], specify_float),
                        coords[2][14]:round(row[102], specify_float),
                        coords[2][15]:round(row[106], specify_float),
                        coords[2][16]:round(row[110], specify_float),
                        coords[2][17]:round(row[114], specify_float),
                        coords[2][18]:round(row[118], specify_float),
                        coords[2][19]:round(row[122], specify_float),
                        coords[2][20]:round(row[126], specify_float),
                        coords[2][21]:round(row[130], specify_float),

                        # v12 to v33
                        coords[3][0]:round(row[47], specify_float),
                        coords[3][1]:round(row[51], specify_float),
                        coords[3][2]:round(row[55], specify_float),
                        coords[3][3]:round(row[59], specify_float),
                        coords[3][4]:round(row[63], specify_float),
                        coords[3][5]:round(row[67], specify_float),
                        coords[3][6]:round(row[71], specify_float),
                        coords[3][7]:round(row[75], specify_float),
                        coords[3][8]:round(row[79], specify_float),
                        coords[3][9]:round(row[83], specify_float),
                        coords[3][10]:round(row[87], specify_float),
                        coords[3][11]:round(row[91], specify_float),
                        coords[3][12]:round(row[95], specify_float),
                        coords[3][13]:round(row[99], specify_float),
                        coords[3][14]:round(row[103], specify_float),
                        coords[3][15]:round(row[107], specify_float),
                        coords[3][16]:round(row[111], specify_float),
                        coords[3][17]:round(row[115], specify_float),
                        coords[3][18]:round(row[119], specify_float),
                        coords[3][19]:round(row[123], specify_float),
                        coords[3][20]:round(row[127], specify_float),
                        coords[3][21]:round(row[131], specify_float),
                    }
                    input_dict = {name: np.expand_dims(np.array(value, dtype=np.float32), axis=0) for name, value in dict_p12_to_p33.items()}
                    
                    # Make Detections
                    # 0: cat_camel, 1: bridge_exercise, 2: heel_raise.
                    result = tflite_inference(input=input_dict, model=model)
                    body_language_class = np.argmax(result)
                    # body_language_prob = round(result[np.argmax(result)], 2)*100
                    body_language_prob = result[np.argmax(result)]

                   
                    if str(body_language_class) == '0':
                        pose_class = 'Cat camel' 
                    elif str(body_language_class) == '1':
                        pose_class = 'Adho mukho Svanasana'
                    elif str(body_language_class) == '2':
                        pose_class = 'Setu bandh'
                    elif str(body_language_class) == '3':
                        pose_class = 'Tadasana'
                    elif str(body_language_class) == '4':
                        pose_class = 'Standing Oblique Crunch'
                    elif str(body_language_class) == '5':
                        pose_class = 'Uttansana'
                    else:
                        pose_class = 'detecting exercise'
                    
                    # print(f'calss: {body_language_class}, prob: {body_language_prob}')

                    # Show pose category near the ear.
                    coords = tuple(np.multiply(
                        np.array(
                            (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                            results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)),
                        [1280,480]
                    ).astype(int))

                    # cv2.rectangle(影像, 頂點座標, 對向頂點座標, 顏色, 線條寬度).
                    # cv2.putText(影像, 文字, 座標, 字型, 大小, 顏色, 線條寬度, 線條種類).
                    cv2.rectangle(image, 
                                (coords[0], coords[1]+5), 
                                (coords[0]+200, coords[1]-30), 
                                (245, 117, 16), -1)
                    cv2.putText(image, pose_class, coords, 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                    # Get status box
                    cv2.rectangle(image, (10,0), (310, 55), (0, 0, 0), -1)

                    # Display Class
                    cv2.putText(
                        image, 
                        'CLASS: ', (15, 25), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.9, (255, 255, 0), 1, cv2.LINE_AA
                    )
                    cv2.putText(
                        image, 
                        pose_class, (120, 25), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        1, (255, 255, 255), 2, cv2.LINE_AA
                    )

                    # Display Probability
                    cv2.putText(
                        image, 
                        'PROB: ', (15, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.9, (255, 255, 0), 1, cv2.LINE_AA
                    )
                    cv2.putText(
                        image, 
                        str(body_language_prob), (120, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        1, (255, 255, 255), 2, cv2.LINE_AA
                    )

                except:
                    pass

                out.write(image)
                cv2.imshow('Pose classification result', image)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            
            else:
                break

    print('Save done!')
    cap.release()
    out.release()
    cv2.destroyAllWindows()


def sample1_xyz(file):
    df = test_data_xyz(file)
    # print(f'df: \n{df}')
    
    coords = point()
    specify_float = 8
    print(round(df.iat[0, 66], specify_float))
    # bb=df['z33']
    # print(f'z33: {bb}')
    
    sample = {
        # x12 to x33
        coords[0][0]:round(df.iat[0, 1], specify_float),
        coords[0][1]:round(df.iat[0, 4], specify_float),
        coords[0][2]:round(df.iat[0, 7], specify_float),
        coords[0][3]:round(df.iat[0, 10], specify_float),
        coords[0][4]:round(df.iat[0, 13], specify_float),
        coords[0][5]:round(df.iat[0, 16], specify_float),
        coords[0][6]:round(df.iat[0, 19], specify_float),
        coords[0][7]:round(df.iat[0, 22], specify_float),
        coords[0][8]:round(df.iat[0, 25], specify_float),
        coords[0][9]:round(df.iat[0, 28], specify_float),
        coords[0][10]:round(df.iat[0, 31], specify_float),
        coords[0][11]:round(df.iat[0, 34], specify_float),
        coords[0][12]:round(df.iat[0, 37], specify_float),
        coords[0][13]:round(df.iat[0, 40], specify_float),
        coords[0][14]:round(df.iat[0, 43], specify_float),
        coords[0][15]:round(df.iat[0, 46], specify_float),
        coords[0][16]:round(df.iat[0, 49], specify_float),
        coords[0][17]:round(df.iat[0, 52], specify_float),
        coords[0][18]:round(df.iat[0, 55], specify_float),
        coords[0][19]:round(df.iat[0, 58], specify_float),
        coords[0][20]:round(df.iat[0, 61], specify_float),
        coords[0][21]:round(df.iat[0, 64], specify_float),

        # y12 to y33
        coords[1][0]:round(df.iat[0, 2], specify_float),
        coords[1][1]:round(df.iat[0, 5], specify_float),
        coords[1][2]:round(df.iat[0, 8], specify_float),
        coords[1][3]:round(df.iat[0, 11], specify_float),
        coords[1][4]:round(df.iat[0, 14], specify_float),
        coords[1][5]:round(df.iat[0, 17], specify_float),
        coords[1][6]:round(df.iat[0, 20], specify_float),
        coords[1][7]:round(df.iat[0, 23], specify_float),
        coords[1][8]:round(df.iat[0, 26], specify_float),
        coords[1][9]:round(df.iat[0, 39], specify_float),
        coords[1][10]:round(df.iat[0, 32], specify_float),
        coords[1][11]:round(df.iat[0, 35], specify_float),
        coords[1][12]:round(df.iat[0, 38], specify_float),
        coords[1][13]:round(df.iat[0, 41], specify_float),
        coords[1][14]:round(df.iat[0, 44], specify_float),
        coords[1][15]:round(df.iat[0, 47], specify_float),
        coords[1][16]:round(df.iat[0, 50], specify_float),
        coords[1][17]:round(df.iat[0, 53], specify_float),
        coords[1][18]:round(df.iat[0, 56], specify_float),
        coords[1][19]:round(df.iat[0, 59], specify_float),
        coords[1][20]:round(df.iat[0, 62], specify_float),
        coords[1][21]:round(df.iat[0, 65], specify_float),

        # z12 to z33
        coords[2][0]:round(df.iat[0, 3], specify_float),
        coords[2][1]:round(df.iat[0, 6], specify_float),
        coords[2][2]:round(df.iat[0, 9], specify_float),
        coords[2][3]:round(df.iat[0, 12], specify_float),
        coords[2][4]:round(df.iat[0, 15], specify_float),
        coords[2][5]:round(df.iat[0, 18], specify_float),
        coords[2][6]:round(df.iat[0, 21], specify_float),
        coords[2][7]:round(df.iat[0, 24], specify_float),
        coords[2][8]:round(df.iat[0, 27], specify_float),
        coords[2][9]:round(df.iat[0, 30], specify_float),
        coords[2][10]:round(df.iat[0, 33], specify_float),
        coords[2][11]:round(df.iat[0, 36], specify_float),
        coords[2][12]:round(df.iat[0, 39], specify_float),
        coords[2][13]:round(df.iat[0, 42], specify_float),
        coords[2][14]:round(df.iat[0, 45], specify_float),
        coords[2][15]:round(df.iat[0, 48], specify_float),
        coords[2][16]:round(df.iat[0, 51], specify_float),
        coords[2][17]:round(df.iat[0, 54], specify_float),
        coords[2][18]:round(df.iat[0, 57], specify_float),
        coords[2][19]:round(df.iat[0, 60], specify_float),
        coords[2][20]:round(df.iat[0, 63], specify_float),
        coords[2][21]:round(df.iat[0, 66], specify_float),
    }

    return sample


def sample2_xyzv(file):
    df = test_data_xyzv(file)
    # print(f'df: \n{df}')
    
    coords = point()
    specify_float = 8
    
    sample = {
        # x12 to x33
        coords[0][0]:round(df.iat[0, 1], specify_float),
        coords[0][1]:round(df.iat[0, 5], specify_float),
        coords[0][2]:round(df.iat[0, 9], specify_float),
        coords[0][3]:round(df.iat[0, 13], specify_float),
        coords[0][4]:round(df.iat[0, 17], specify_float),
        coords[0][5]:round(df.iat[0, 21], specify_float),
        coords[0][6]:round(df.iat[0, 25], specify_float),
        coords[0][7]:round(df.iat[0, 29], specify_float),
        coords[0][8]:round(df.iat[0, 33], specify_float),
        coords[0][9]:round(df.iat[0, 37], specify_float),
        coords[0][10]:round(df.iat[0, 41], specify_float),
        coords[0][11]:round(df.iat[0, 45], specify_float),
        coords[0][12]:round(df.iat[0, 49], specify_float),
        coords[0][13]:round(df.iat[0, 53], specify_float),
        coords[0][14]:round(df.iat[0, 57], specify_float),
        coords[0][15]:round(df.iat[0, 61], specify_float),
        coords[0][16]:round(df.iat[0, 65], specify_float),
        coords[0][17]:round(df.iat[0, 69], specify_float),
        coords[0][18]:round(df.iat[0, 73], specify_float),
        coords[0][19]:round(df.iat[0, 77], specify_float),
        coords[0][20]:round(df.iat[0, 81], specify_float),
        coords[0][21]:round(df.iat[0, 85], specify_float),

        # y12 to y33
        coords[1][0]:round(df.iat[0, 2], specify_float),
        coords[1][1]:round(df.iat[0, 6], specify_float),
        coords[1][2]:round(df.iat[0, 10], specify_float),
        coords[1][3]:round(df.iat[0, 14], specify_float),
        coords[1][4]:round(df.iat[0, 18], specify_float),
        coords[1][5]:round(df.iat[0, 22], specify_float),
        coords[1][6]:round(df.iat[0, 26], specify_float),
        coords[1][7]:round(df.iat[0, 30], specify_float),
        coords[1][8]:round(df.iat[0, 34], specify_float),
        coords[1][9]:round(df.iat[0, 38], specify_float),
        coords[1][10]:round(df.iat[0, 42], specify_float),
        coords[1][11]:round(df.iat[0, 46], specify_float),
        coords[1][12]:round(df.iat[0, 50], specify_float),
        coords[1][13]:round(df.iat[0, 51], specify_float),
        coords[1][14]:round(df.iat[0, 58], specify_float),
        coords[1][15]:round(df.iat[0, 62], specify_float),
        coords[1][16]:round(df.iat[0, 66], specify_float),
        coords[1][17]:round(df.iat[0, 70], specify_float),
        coords[1][18]:round(df.iat[0, 74], specify_float),
        coords[1][19]:round(df.iat[0, 78], specify_float),
        coords[1][20]:round(df.iat[0, 82], specify_float),
        coords[1][21]:round(df.iat[0, 86], specify_float),

        # z12 to z33
        coords[2][0]:round(df.iat[0, 3], specify_float),
        coords[2][1]:round(df.iat[0, 7], specify_float),
        coords[2][2]:round(df.iat[0, 11], specify_float),
        coords[2][3]:round(df.iat[0, 15], specify_float),
        coords[2][4]:round(df.iat[0, 19], specify_float),
        coords[2][5]:round(df.iat[0, 23], specify_float),
        coords[2][6]:round(df.iat[0, 27], specify_float),
        coords[2][7]:round(df.iat[0, 31], specify_float),
        coords[2][8]:round(df.iat[0, 35], specify_float),
        coords[2][9]:round(df.iat[0, 39], specify_float),
        coords[2][10]:round(df.iat[0, 43], specify_float),
        coords[2][11]:round(df.iat[0, 47], specify_float),
        coords[2][12]:round(df.iat[0, 51], specify_float),
        coords[2][13]:round(df.iat[0, 55], specify_float),
        coords[2][14]:round(df.iat[0, 59], specify_float),
        coords[2][15]:round(df.iat[0, 63], specify_float),
        coords[2][16]:round(df.iat[0, 67], specify_float),
        coords[2][17]:round(df.iat[0, 71], specify_float),
        coords[2][18]:round(df.iat[0, 75], specify_float),
        coords[2][19]:round(df.iat[0, 79], specify_float),
        coords[2][20]:round(df.iat[0, 83], specify_float),
        coords[2][21]:round(df.iat[0, 87], specify_float),

        # v12 to v33
        coords[3][0]:round(df.iat[0, 4], specify_float),
        coords[3][1]:round(df.iat[0, 8], specify_float),
        coords[3][2]:round(df.iat[0, 12], specify_float),
        coords[3][3]:round(df.iat[0, 16], specify_float),
        coords[3][4]:round(df.iat[0, 20], specify_float),
        coords[3][5]:round(df.iat[0, 24], specify_float),
        coords[3][6]:round(df.iat[0, 28], specify_float),
        coords[3][7]:round(df.iat[0, 32], specify_float),
        coords[3][8]:round(df.iat[0, 36], specify_float),
        coords[3][9]:round(df.iat[0, 40], specify_float),
        coords[3][10]:round(df.iat[0, 44], specify_float),
        coords[3][11]:round(df.iat[0, 48], specify_float),
        coords[3][12]:round(df.iat[0, 52], specify_float),
        coords[3][13]:round(df.iat[0, 56], specify_float),
        coords[3][14]:round(df.iat[0, 60], specify_float),
        coords[3][15]:round(df.iat[0, 64], specify_float),
        coords[3][16]:round(df.iat[0, 68], specify_float),
        coords[3][17]:round(df.iat[0, 72], specify_float),
        coords[3][18]:round(df.iat[0, 76], specify_float),
        coords[3][19]:round(df.iat[0, 80], specify_float),
        coords[3][20]:round(df.iat[0, 84], specify_float),
        coords[3][21]:round(df.iat[0, 88], specify_float),
    }

    return sample


def test_data_xyz(file):
    df = pd.read_csv(file)
    df2 = df.copy()

    columns_removed = [
        'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11',
        'y1', 'y2', 'y3', 'y4', 'y5', 'y6', 'y7', 'y8', 'y9', 'y10', 'y11',
        'z1', 'z2', 'z3', 'z4', 'z5', 'z6', 'z7', 'z8', 'z9', 'z10', 'z11',
        'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11',

        'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19', 'v20', 'v21',
        'v22', 'v23', 'v24', 'v25', 'v26', 'v27', 'v28', 'v29', 'v30', 'v31',
        'v32', 'v33',
    ]

    df2 = df2.drop(columns_removed, axis = 'columns')

    # Get A row from A to B.
    get_a_row_value = df2.iloc[4:5]
    return get_a_row_value


def test_data_xyzv(file):
    df = pd.read_csv(file)
    df2 = df.copy()

    columns_removed = [
        'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11',
        'y1', 'y2', 'y3', 'y4', 'y5', 'y6', 'y7', 'y8', 'y9', 'y10', 'y11',
        'z1', 'z2', 'z3', 'z4', 'z5', 'z6', 'z7', 'z8', 'z9', 'z10', 'z11',
        'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11',
    ]

    df2 = df2.drop(columns_removed, axis = 'columns')

    # Get row from 4 to 5.
    get_a_row_value = df2.iloc[4:5]
    return get_a_row_value


def desktop_model_inference(input, model):
    '''# Loads the model and training weights for desktop model and test inference.'''

    model = keras.models.load_model(model)
    # print(model.summary())

    # print(f'input: \n{type(input)}')
    outputs = model.predict(input)

    # print('*'*30) 
    # print(f'tatal: {outputs[0][0] + outputs[0][1] + outputs[0][2]}')
    # print(f'calss: {np.argmax(outputs[0])}, prob: {outputs[0][np.argmax(outputs[0])]}')
    # print(f'calss: {np.argmax(outputs[0])}, prob: {round(outputs[0][np.argmax(outputs[0])]*100, 5)}%')
    return outputs


def tflite_inference(input, model):
    # Load TFLite model and allocate tensors.
    interpreter = tf.lite.Interpreter(model_path=model)
    interpreter.allocate_tensors()

    ### Get input and output tensors. ###
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # tf.print('input_details[0]:\n', input_details[0])
    # tf.print('output_details:\n', output_details)
    # print('-'*30)
    # print(f'input_index: {input_details[0]["index"]}')
    # print(f'output_index: {output_details[0]["index"]}')
    # print(f'input_shape: {input_details[0]["shape"]}')
    # print('-'*30)

    ### Verify the TensorFlow Lite model. ###
    for i, (name, value) in enumerate(input.items()):

        input_value = np.expand_dims(value, axis=1)
        # print(f'index: {i}, type: {type(input_value)}, shape:{input_value.shape}')
        # print(input_value)
        interpreter.set_tensor(input_details[i]['index'], input_value)
        interpreter.invoke()

    output = interpreter.tensor(output_details[0]['index'])()[0]

    # print(f'prob: {output}, type: {type(output)}')
    # print(f'calss: {np.argmax(output)}, prob: {output[np.argmax(output)]}')
    return output


def test_model_inference(input_csv, pc_model, tflite_model):
    '''Input a row data(point12 to point33 features) to desktop model and tflite model from test csv file for inference.'''
    test_input_xyzv = sample2_xyzv(file=input_csv)
    input_dict = {name: np.expand_dims(np.array(value, dtype=np.float32), axis=0) for name, value in test_input_xyzv.items()}

    result_pc_model = desktop_model_inference(input=input_dict, model=pc_model)
    result_tflite = tflite_inference(input=input_dict, model=tflite_model)

    print('-'*30)
    print(f'[Desktop model inference]\nprob: {result_pc_model}, type: {type(result_pc_model)}')
    print(f'calss: {np.argmax(result_pc_model[0])}, prob: {result_pc_model[0][np.argmax(result_pc_model[0])]}')
    print('-'*30)
    print(f'[TFLite model inference]\nprob: {result_tflite}, type: {type(result_tflite)}')
    print(f'calss: {np.argmax(result_tflite)}, prob: {result_tflite[np.argmax(result_tflite)]}')
    print('-'*30)


if __name__ == '__main__':

    # 0: cat_camel, 1: bridge_exercise, 2: heel_raise
    video_file_name = '5.uttansana'
    output_video = '' + video_file_name + 'my_out.mp4'
    video_path = '../video/'+ video_file_name + '.mp4'
    
    test_file = './datasets/numerical_coords_dataset_test2.csv'
    all_model = './model_weights/all_model/08.31_xyzv/3_categories_pose'
    tflite_model = 'model.tflite'

    cap = cv2.VideoCapture(video_path)

    # test_model_inference(input_csv=test_file, pc_model=all_model, tflite_model=tflite_model)

    display_tflite_classify_pose(cap, model=tflite_model)
    # save_tflite_classify_pose(cap, model=tflite_model, result_out=output_video)

Frames per second: 25.0
Frame count: 640
calss: 2, prob: 0.9959879517555237
calss: 2, prob: 0.998489260673523
calss: 2, prob: 0.9980543851852417
calss: 2, prob: 0.9981676340103149
calss: 2, prob: 0.9978145360946655
calss: 2, prob: 0.9975318908691406
calss: 2, prob: 0.9975684285163879
calss: 2, prob: 0.9976621866226196
calss: 2, prob: 0.9980455636978149
calss: 2, prob: 0.9984455704689026
calss: 2, prob: 0.9986048340797424
calss: 2, prob: 0.9987194538116455
calss: 2, prob: 0.9989043474197388
calss: 2, prob: 0.999025821685791
calss: 2, prob: 0.999144434928894
calss: 2, prob: 0.9991766810417175
calss: 2, prob: 0.9991589784622192
calss: 2, prob: 0.9991594552993774
calss: 2, prob: 0.9991430044174194
calss: 2, prob: 0.9991249442100525
calss: 2, prob: 0.9991413354873657
calss: 2, prob: 0.999157190322876
calss: 2, prob: 0.9991769194602966
calss: 2, prob: 0.9991965889930725
calss: 2, prob: 0.9992159605026245
calss: 2, prob: 0.9992300271987915
calss: 2, prob: 0.9992382526397705
calss: 2, prob: 0.

Frames per second: 25.0
Frame count: 645
Done!
Error opening the video file.
video_w: 0, video_h: 0


UnboundLocalError: local variable 'output_fps' referenced before assignment